In [29]:
import os
import mysql.connector
import pandas as pd
from google.cloud import bigquery

#variables
curr_path = os.getcwd()
load_file = 'employee_salary.csv'
load_file = os.path.join(curr_path,load_file)


proj = 'dark-arcade-431219-f6'
dataset = 'sample_database'
target_table = 'annual_employee_salary'
table_id = f'{proj}.{dataset}.{target_table}'

# data connections
conn = mysql.connector.connect(host='localhost',
                                user='root',
                                passwd='hasibjan1'
                              )

client = bigquery.Client(project=proj)

mycursor = conn.cursor()
print(conn)

In [9]:
#Create our SQL extract query
mycursor.execute('SELECT e.emp_no,e.birth_date,e.first_name,e.last_name,s.salary,AVG(salary) OVER(PARTITION BY emp_no) AS EmpSalary FROM employees.employees e JOIN employees.salaries s ON e.emp_no = s.emp_no')

output = mycursor.fetchall()

for x in output:
    print(x)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
#Extract data in data frame
sql = 'SELECT e.emp_no,e.birth_date,e.first_name,e.last_name,s.salary,AVG(salary) OVER(PARTITION BY emp_no) AS EmpSalary FROM employees.employees e JOIN employees.salaries s ON e.emp_no = s.emp_no'

df = pd.read_sql(sql,conn)
df

/var/folders/rb/0gt4r16106z3q6h2ky_t2d900000gn/T/ipykernel_8592/3515389645.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,conn)


,emp_no,birth_date,first_name,last_name,salary,EmpSalary
0,10001,1953-09-02,Georgi,Facello,60117,75388.9412
1,10001,1953-09-02,Georgi,Facello,62102,75388.9412
2,10001,1953-09-02,Georgi,Facello,66074,75388.9412
3,10001,1953-09-02,Georgi,Facello,66596,75388.9412
4,10001,1953-09-02,Georgi,Facello,66961,75388.9412
...,...,...,...,...,...,...
967325,201771,1962-12-24,Fumitaka,Gammage,56736,54893.1538
967326,201771,1962-12-24,Fumitaka,Gammage,56577,54893.1538
967327,201771,1962-12-24,Fumitaka,Gammage,60056,54893.1538
967328,201771,1962-12-24,Fumitaka,Gammage,62899,54893.1538


In [25]:
#Export as csv
df.to_csv('/Users/hasibazami/desktop/etl_data_folder/employee_salary.csv')

In [11]:
def salaries(r):
    if r >= 5000:
        return 'over 500k'
    elif r <= 5000:
        return 'less than 500k'


In [30]:
#Load
job_config = bigquery.LoadJobConfig(
    skip_leading_rows=1,
    source_format=bigquery.SourceFormat.CSV,
    autodetect=True,
    write_disposition='WRITE_TRUNCATE'
)

#open file for loading
with open(load_file,'rb') as file:
    load_job = client.load_table_from_file(
        file,
        table_id,
        job_config=job_config
    )

load_job.result()

#Check how many records were loaded
dest_table = client.get_table(table_id)
print(f"you have {dest_table.num_rows} in your table")

you have 967330 in your table


In [18]:
print(os.chdir('/users/hasibazami/Desktop/etl_data_folder'))

None
